**The Battle of Neighborhoods Week 2 - Finding the right neighborhood to live in New York City**

*INTRODUCTION*

**Background**

People who want to come to New York City always ask the same question. What neighborhood is best to eat in New York City? My question back to them is what type of food are you looking to eat? New York City is has a very diverse food culture. I cannot say that one neighborhood’s food is better than another because it depends on the individual’s preference or craving at the moment. For example, if you are craving for pizza, there are many options in each neigborhoods. It depends if you are looking for a pizza place in your neighborhood or if you are willing to travel to a different neighborhood for pizza place that have a better rating.

This capstone project is to help people find places to eat in any neighbhorhood in New York City. It will also help those who are visiting New York City to decide which neighborhood to stay depending on the surrounding restaurants and cafes nearby. 

Also people who are already living in New York City will benefit from this project as well because just because you live in the city that does not mean you know all the neighborhood and what kind of venues each neighborhood has to offer. The project will people explore the surrounding areas and see what kind of venues are there so they can make the best choice that best fits their needs. 

**Problem**

There are many expat groups in New York City and a lot more people are coming to live in New York City. Each year New York City receive a great amount of tourist all over the world looking for places to eat around them and while they are traveling. For those that live in New York City already, sometime when they change neighborhood, they become unfamiliar with their environment so it takes time for them to know where to go and eat.

The more information we can provide to help people become more aware of the surrounding venues in each neighborhood, the more prepared they will be before visiting or moving to New York City.

**Interest**

This project is also a personal interest to me because I used to live in New York City but I do not know all the neighborhoods and what kind of venues it entails. This is why I mentioned that just because you live in New York City, that does not mean you know what each neighborhood has to offer.

Also, I can share this information with expat communities, friends that planning to visit New York City, and others that would be interested in having this information.

*DATA*

The data I will collect is similar to what we have been doing from the course materials. I will use the data to convert addresses into their equivalent latitude and longitude values. Also, I will use the Foursquare API to explore neighborhoods in New York City. Then I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the k-means clustering algorithm to complete this task and I will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

By using all these different methods to make the data more understandable. I can make an analysis of what type of venues each neighborhood in New York City has in order to help those who need help in deciding where to live in New York City.

*METHODOLOGY*

**1. Download and explore dataset**

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, I will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

This dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

Since the file is already on the server, I can simply run a wget command and access the data. 

To get started I need to download all of the dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Run a wget command to access the data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data and take a look at the first item in this list.

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

**Tranform the data into a pandas dataframe**

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe. Then take a look at the empty dataframe to confirm that the columns are as intended.

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time and examine the resulting dataframe.

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


**Define Foursquare Credentials and Version**

In [8]:
CLIENT_ID = 'ZCI4OEQHMY5MRLGZFJLQECL35KXS5F4DISXJPB2E2YLJMB5H' # your Foursquare ID
CLIENT_SECRET = 'N1ZVVSG4NSRFPJOPZ3ZZBBC5H4XKSRHP3EGKRCL5K1UDR1IC' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZCI4OEQHMY5MRLGZFJLQECL35KXS5F4DISXJPB2E2YLJMB5H
CLIENT_SECRET:N1ZVVSG4NSRFPJOPZ3ZZBBC5H4XKSRHP3EGKRCL5K1UDR1IC


Exploring Neighhorhoods in New York City

In [9]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Explore nearby venues in the neighborhoods

In [10]:
newyorkcity_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

First 5 venues with latitue and longitude

In [11]:
newyorkcity_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


Neighborhood counts of venues.

In [12]:
newyorkcity_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,29,29,29,29,29,29
Annadale,9,9,9,9,9,9
Arden Heights,5,5,5,5,5,5
Arlington,5,5,5,5,5,5
Arrochar,23,23,23,23,23,23
Arverne,20,20,20,20,20,20
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,20,20,20,20,20,20


Analysing each Neighborhood

In [13]:
# one hot encoding
nyc_onehot = pd.get_dummies(newyorkcity_venues[['Venue Category']], prefix="", prefix_sep="")
nyc_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
nyc_onehot.insert(loc=0, column='Neighborhood', value=newyorkcity_venues['Neighborhood'] )
nyc_onehot.shape

(10119, 428)

Group the neighborhood

In [14]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery

Return most common venues.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Top 10 most common venues in each neighborhood.

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Spa,Supermarket,Donut Shop,Fast Food Restaurant,Bus Station,Martial Arts Dojo,Bakery
1,Annadale,Dance Studio,Food,Bar,Train Station,American Restaurant,Diner,Restaurant,Pizza Place,Cosmetics Shop,Filipino Restaurant
2,Arden Heights,Deli / Bodega,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Film Studio,Eye Doctor,Factory,Falafel Restaurant,Farm
3,Arlington,Deli / Bodega,Intersection,American Restaurant,Bus Stop,Grocery Store,Fish & Chips Shop,Factory,Falafel Restaurant,Farm,Farmers Market
4,Arrochar,Deli / Bodega,Pizza Place,Bagel Shop,Bus Stop,Italian Restaurant,Middle Eastern Restaurant,Sandwich Place,Liquor Store,Outdoors & Recreation,Mediterranean Restaurant


Run k-means to cluster the neighborhood into 5 clusters

In [17]:
# set number of clusters
kclusters = 5

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0,
       2, 0, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 2, 1, 0, 0, 2, 2, 0, 0, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2,
       2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0,
       2, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 4, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,
       0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 1, 2, 2,
       0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0,
       0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0,
       0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 1,
       0, 2, 0, 1, 2, 2, 2, 2, 3, 0, 1, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2,
       0, 2, 0, 2, 2, 3, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0,
       2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0,

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

nyc_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Donut Shop,Gas Station,Laundromat,Ice Cream Shop,Sandwich Place,Dessert Shop,Deli / Bodega,Factory,Falafel Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,0,Bus Station,Baseball Field,Pizza Place,Supermarket,Fast Food Restaurant,Grocery Store,Park,Basketball Court,Bagel Shop,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,0,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Automotive Shop,Bowling Alley,Bakery,Chinese Restaurant,Pizza Place,Convenience Store
3,Bronx,Fieldston,40.895437,-73.905643,0,Plaza,Music Venue,River,Bus Station,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Baseball Field,Bank,Bus Station,Gym,Home Service,Plaza,Film Studio,Eye Doctor,Factory


Neighborhood venues sorted.

In [19]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Spa,Supermarket,Donut Shop,Fast Food Restaurant,Bus Station,Martial Arts Dojo,Bakery
1,2,Annadale,Dance Studio,Food,Bar,Train Station,American Restaurant,Diner,Restaurant,Pizza Place,Cosmetics Shop,Filipino Restaurant
2,0,Arden Heights,Deli / Bodega,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Film Studio,Eye Doctor,Factory,Falafel Restaurant,Farm
3,2,Arlington,Deli / Bodega,Intersection,American Restaurant,Bus Stop,Grocery Store,Fish & Chips Shop,Factory,Falafel Restaurant,Farm,Farmers Market
4,2,Arrochar,Deli / Bodega,Pizza Place,Bagel Shop,Bus Stop,Italian Restaurant,Middle Eastern Restaurant,Sandwich Place,Liquor Store,Outdoors & Recreation,Mediterranean Restaurant


**Cluster 1**

In [20]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Donut Shop,Gas Station,Laundromat,Ice Cream Shop,Sandwich Place,Dessert Shop,Deli / Bodega,Factory,Falafel Restaurant
1,Co-op City,Bus Station,Baseball Field,Pizza Place,Supermarket,Fast Food Restaurant,Grocery Store,Park,Basketball Court,Bagel Shop,Pharmacy
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Automotive Shop,Bowling Alley,Bakery,Chinese Restaurant,Pizza Place,Convenience Store
3,Fieldston,Plaza,Music Venue,River,Bus Station,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Riverdale,Park,Baseball Field,Bank,Bus Station,Gym,Home Service,Plaza,Film Studio,Eye Doctor,Factory
5,Kingsbridge,Pizza Place,Bar,Sandwich Place,Latin American Restaurant,Liquor Store,Mexican Restaurant,Supermarket,Donut Shop,Bakery,Spanish Restaurant
6,Marble Hill,Coffee Shop,Gym,Yoga Studio,Big Box Store,Miscellaneous Shop,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Sandwich Place
7,Woodlawn,Pizza Place,Deli / Bodega,Pub,Playground,Cosmetics Shop,Grocery Store,Park,Food Truck,Bakery,Trail
8,Norwood,Pizza Place,Bus Station,Bank,Chinese Restaurant,Park,Pharmacy,Convenience Store,Sandwich Place,Mobile Phone Shop,Deli / Bodega
10,Baychester,Donut Shop,Fried Chicken Joint,Convenience Store,Mattress Store,Mexican Restaurant,Baseball Field,Fast Food Restaurant,Bank,Pet Store,Electronics Store


**Cluster 2**

In [21]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,Coney Island,Baseball Stadium,Beach,Café,Theme Park Ride / Attraction,Skating Rink,Caribbean Restaurant,Vegetarian / Vegan Restaurant,Monument / Landmark,Music Venue,Food Court
85,Sea Gate,Bus Station,Beach,American Restaurant,Spa,Construction & Landscaping,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market
172,Breezy Point,Beach,Monument / Landmark,Trail,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
178,Rockaway Beach,Beach,Ice Cream Shop,BBQ Joint,Deli / Bodega,Arepa Restaurant,Latin American Restaurant,Seafood Restaurant,Supermarket,Beach Bar,New American Restaurant
179,Neponsit,Beach,Bar,Lounge,Yoga Studio,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
190,Belle Harbor,Beach,Pub,Spa,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Bakery,Bagel Shop,Donut Shop,Chinese Restaurant
191,Rockaway Park,Beach,Pizza Place,Donut Shop,Steakhouse,Seafood Restaurant,French Restaurant,Bar,Bank,Bagel Shop,Board Shop
193,Brookville,Deli / Bodega,Recording Studio,Yoga Studio,Fish & Chips Shop,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
204,South Beach,Deli / Bodega,Beach,Pier,Bus Stop,Athletics & Sports,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
232,Midland Beach,Deli / Bodega,Beach,Bus Stop,Restaurant,Liquor Store,Basketball Court,Bookstore,Café,Pet Store,Dessert Shop


**Cluster 3**

In [22]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Williamsbridge,Nightclub,Soup Place,Caribbean Restaurant,Bar,Fountain,Film Studio,Eye Doctor,Factory,Falafel Restaurant,Farm
12,City Island,Grocery Store,Seafood Restaurant,Harbor / Marina,Thrift / Vintage Store,Deli / Bodega,History Museum,Café,French Restaurant,Music Venue,Smoke Shop
27,Clason Point,Park,Boat or Ferry,South American Restaurant,Recording Studio,Bus Stop,Pool,Convenience Store,Grocery Store,Factory,Falafel Restaurant
28,Throgs Neck,Coffee Shop,Italian Restaurant,Asian Restaurant,Deli / Bodega,Sports Bar,Baseball Field,Pizza Place,Bar,American Restaurant,Distillery
34,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Bank,Dessert Shop,Grocery Store,Donut Shop,Fish Market,Coffee Shop
39,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Coffee Shop,Sports Bar,Japanese Restaurant,Pub,American Restaurant,Park,Spa
46,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bagel Shop,Chinese Restaurant,Bar,American Restaurant,Sandwich Place,Grocery Store
49,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Yoga Studio,Sushi Restaurant,Mexican Restaurant,French Restaurant,Café,Bakery
52,Sheepshead Bay,Turkish Restaurant,Dessert Shop,Sandwich Place,Yoga Studio,Diner,Creperie,Restaurant,Pizza Place,Russian Restaurant,Outlet Store
58,Windsor Terrace,Diner,Deli / Bodega,Plaza,Park,Café,Grocery Store,Bagel Shop,Chinese Restaurant,Butcher,Sushi Restaurant


**Cluster 4**

In [23]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 3, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Yoga Studio,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
203,Todt Hill,Park,Yoga Studio,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


**Cluster 5**

In [24]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 4, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
202,Grymes Hill,Dog Run,Deli / Bodega,Yoga Studio,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


**Getting the cordinates for New York City**

In [25]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


**Creaping a map of New York City**

In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

*REPORTS*

For this project I used the data from this link: https://geo.nyu.edu/catalog/nyu_2451_34572 And once the data was downloaded, I opened the json file of the date and transform it into a pandas dataframe by inputing all the necessary information to create a dataframe which consists of New York City: Borough, Neighborhood, Latitude, and Longitude.

Once I was satisfed with the dataframe, I called the Foursquare API to start preparing to get the dataset of venues in New York City's neighborhood. Then I explore each neighborhoods and venues with their latitude and longitude. Since there were many neighbords and venues, I did a count to see how many venues in each neighbhood, then I further analyse the data by each neighborhood and find the most common venues and limit that to the top 10 venues for each neighborhood. 

Then I ran k-mean to cluster 5 neighborhoods, then I ran each cluster individually to see what kind of results it yield. 

Lastly, I use the geolocator to get New York City's latitude and longitude, then created a map of New York City with different color markers. 

*DISCUSSION*

**Result**

The data from New York University catalog that we used to get New York City's boroughs, neighborhoods, latitude, and longitude showed that we have 5 boroughs and 306 neighborhoods. 

The data from Foursquare shows all the venues in each neighbhorhoods which yield many results that I had to only limit it to the top 10 venues for each neighborhoods and created 5 clusters of the neighborhoods.

The latitude and longitude of New York City is 40.7127281, -74.0060152.

The map of New York City shows the 5 markers with different color to identify the different cluster. The colors are: Red = Cluster 1 (clustser 0 on map), Purple = Cluster 2 (cluster 1 on map), Blue = Cluster 3 (cluster 2 on map), Green = Cluster 4 (cluster 3 on map), Orange = Cluster 5 (cluster 4 on map).

**Observation**

From the data, it shows that New York City has many neighborhoods that are enriched with many different venues.  

In the map,  cluster 1 (cluster 0, ), cluster 2 (cluster1, ) and cluster 3 (cluster 2,) has the most venues in their area.

**Recommendation**

If you are visiting New York City or moving to different neighborhood, the best borough to stay that have a lot of venues are within Manhattan and Brooklyn. There you cannot go wrong in finding any venues that suit your needs since the neighborhoods in those two boroughs have the most venues which you will find something that satisfy you. 

*CONCLUSION*

New York City is a very diverse place full of culture and food. Every neighborhoods are full of different restaurants, cafes, bars, etc for you to choose from and they are open early to accommodate early birds and close at mucher later time to accommodate those who work graveyard shift or for those who are just coming out of the clubs or bars. 

So no matter what neighborhood you are in at anytime of day or night, there will always be venue that is open to accept you when you are hungry.